In [114]:
import pandas as pd
from pandas import Series, DataFrame
import re
import numpy as np
from requests import get
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import time
from mod import *
from datetime import date,timedelta

In [2]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}

In [51]:
chrome_path = 'C:/Users/Ilhoon/Desktop/dev_util/chromedriver_win32/chromedriver'
driver = webdriver.Chrome(chrome_path)

In [3]:
base_db=pd.read_csv('base_product_DB.csv')

In [5]:
score_dict = {'worst': 1, 'bad': 2, 'soso': 3, 'good': 4, 'best':5}

In [119]:
pid=[] # 제품 id
user_name=[] # 유저 이름
age=[] # 나이
skin_type=[] # 피부 타입
d_date=[] # 날짜
text=[] # 리뷰 텍스트
score=[] # 점수

In [63]:
chrome_path = 'C:/Users/Ilhoon/Desktop/dev_util/chromedriver_win32/chromedriver'
options=webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36')
options.add_argument("disable-gpu")
driver = webdriver.Chrome(chrome_path, chrome_options=options)

In [130]:
# 글로우픽 실시간 리뷰
def glowpick_review_crawl():
    for p_id in base_db["pid"]:
        driver.get('http://glowpick.com/product/'+str(p_id)+'')
        html = driver.page_source
        soup = bs(html, 'lxml')

        t_date=datetime.today().strftime('%Y-%m-%d') # 오늘 
        yesterday = date.today() - timedelta(1)
        y_date=yesterday.strftime('%Y-%m-%d') # 어제

        for k in range(len(soup.select('section.section-list-item > ul.review-list-wrap > li'))):
            r_date=re.sub("[^0-9]","-",date_transfer(soup.select('section.section-list-item > ul.review-list-wrap > li > span.date')[k].text))   
            if r_date==t_date or r_date==y_date: # 리뷰 날짜가 오늘이나 어제와 같다면
                # 제품 id 
                pid.append(p_id)
                # 피부타입
                skin_type.append(re.sub("·","",soup.select('section.section-list-item > ul.review-list-wrap > li > div > div > div > p > span.info > span.txt')[k].text[4:]).strip())
                # 나이
                age.append(soup.select('section.section-list-item > ul.review-list-wrap > li > div > div > div > p > span.info > span.txt')[k].text[:3])
                # 유저 이름
                user_name.append(soup.select('section.section-list-item > ul.review-list-wrap > li > div > div > div > p > span.name')[k].text)
                # 리뷰 텍스트
                text.append(soup.select('section.section-list-item > ul.review-list-wrap > li > div > p')[k].text)
                # 날짜 정보
                d_date.append(r_date)
                # 점수
                score.append(score_dict[soup.select('div > div > div > p > span.info > span.label > span')[k*2]['class'][1][4:]])

In [129]:
data={'pid':pid,
     'user_name':user_name,
     'age':age,
     'skin_type':skin_type,
     'date':d_date,
     'text':text,
      'score':score}
gl_new_review=DataFrame(data)
gl_new_review=DataFrame(data, columns=['pid','user_name','age','skin_type','date','text','score'])
gl_new_review

,pid,user_name,age,skin_type,date,text,score
0,94782,수분필요행,27세,건성,2018-05-25,겨울에 정말 잘 사용했던 제품입니다~커버력은 쿠션치고는 괜찮은 편이었고 촉촉해서 건...,4
1,94782,아이싱듬뿍,27세,지성,2018-05-25,커버도 좋구 광도 좋구 컬러도 예쁘고 나스쿠션이 저는 제일 좋아요,5
2,94782,tu7882,19세,민감성,2018-05-25,너무 핑크..그래서 그런지 상아색이 있으면 하는 아쉬움이 남는다,4
3,102506,수분필요행,27세,건성,2018-05-25,"제가 써본 파데 중 피부가 가장 좋아보이고 매끈하게 발립니다 굿굿👍👍👍입생*랑, 조...",5
4,102506,망개뜩,22세,지성,2018-05-25,별명이 결광 파데인 만큼 피부결이 좋아보이고 소량임에도 커버력이 좋다 다만 지성인 ...,4
5,102506,미유S2,25세,건성,2018-05-25,평소 17~19호 사용하는데 몽블랑컬러로 샀어요.그랬더니 민낯보다 어두워지네요. 밝...,4
6,94782,수분필요행,27세,건성,2018-05-25,겨울에 정말 잘 사용했던 제품입니다~커버력은 쿠션치고는 괜찮은 편이었고 촉촉해서 건...,4
7,94782,아이싱듬뿍,27세,지성,2018-05-25,커버도 좋구 광도 좋구 컬러도 예쁘고 나스쿠션이 저는 제일 좋아요,5
8,94782,tu7882,19세,민감성,2018-05-25,너무 핑크..그래서 그런지 상아색이 있으면 하는 아쉬움이 남는다,4


In [177]:
gl_review.to_csv('gl_review.csv',encoding="UTF-8")